In [ ]:
import numpy as np

#Dataset

In [ ]:
# English Sentences
english_sentences = ["Birds eat seeds", "Seeds grow plants"]

# Urdu Sentences
urdu_sentences = ["پرندے بیج کھاتے", "بیج پودے اگاتے"]


#Tokenization

In [ ]:
from transformers import MBart50TokenizerFast
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50")


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
english_tokens = tokenizer(english_sentences, padding=True, truncation=True, return_tensors="pt")
urdu_tokens = tokenizer(urdu_sentences, padding=True, truncation=True, return_tensors="pt")


In [ ]:
# Loop through English sentences
print("English Sentences:\n")
for i, sentence in enumerate(english_tokens["input_ids"]):
    token_ids = sentence.tolist()  # Convert tensor to list
    tokens = tokenizer.convert_ids_to_tokens(token_ids)  # Convert to actual words
    print(f"Sentence {i+1} (Token IDs): {token_ids}")
    print(f"Sentence {i+1} (Tokens): {tokens}\n")

# Loop through Urdu sentences
print("\nUrdu Sentences:\n")
for i, sentence in enumerate(urdu_tokens["input_ids"]):
    token_ids = sentence.tolist()  # Convert tensor to list
    tokens = tokenizer.convert_ids_to_tokens(token_ids)  # Convert to actual words
    print(f"Sentence {i+1} (Token IDs): {token_ids}")
    print(f"Sentence {i+1} (Tokens): {tokens}\n")


English Sentences:

Sentence 1 (Token IDs): [250004, 72607, 7, 73203, 40, 297, 7, 2]
Sentence 1 (Tokens): ['en_XX', '▁Bird', 's', '▁eat', '▁se', 'ed', 's', '</s>']

Sentence 2 (Token IDs): [250004, 6872, 6468, 55993, 18153, 7, 2, 1]
Sentence 2 (Tokens): ['en_XX', '▁See', 'ds', '▁grow', '▁plant', 's', '</s>', '<pad>']


Urdu Sentences:

Sentence 1 (Token IDs): [250004, 523, 1787, 1597, 2547, 2048, 104622, 8012, 2]
Sentence 1 (Tokens): ['en_XX', '▁پر', 'ند', 'ے', '▁بی', 'ج', '▁کھا', 'تے', '</s>']

Sentence 2 (Token IDs): [250004, 2547, 2048, 14013, 136372, 878, 30928, 8012, 2]
Sentence 2 (Tokens): ['en_XX', '▁بی', 'ج', '▁پو', 'دے', '▁ا', 'گا', 'تے', '</s>']



#Input Embedding

In [ ]:
# Check vocabulary size from tokenizer
vocab_size = tokenizer.vocab_size
print("Tokenizer Vocabulary Size:", vocab_size)

Tokenizer Vocabulary Size: 250054


In [ ]:
import torch
import torch.nn as nn

# Function to convert tokenized input IDs into embeddings.
def get_embeddings(input_ids, embedding_dim,vocab_size):

    # Define the embedding layer
    embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

    # Get embeddings
    embedded_tokens = embedding_layer(input_ids)

    return embedded_tokens

In [ ]:
# Example usage:
embedd_dim=3
eng_embeddings=get_embeddings(english_tokens['input_ids'],embedd_dim,vocab_size)
urdu_embeddings=get_embeddings(urdu_tokens['input_ids'],embedd_dim,vocab_size)
print("Shape of embeddings:", eng_embeddings.shape)  # (1, seq_length, embedding_dim)
print("Shape of embeddings:", urdu_embeddings.shape)  # (1, seq_length, embedding_dim)

Shape of embeddings: torch.Size([2, 8, 3])
Shape of embeddings: torch.Size([2, 9, 3])


In [ ]:
print(eng_embeddings)

tensor([[[-0.1074,  2.0117, -0.6662],
         [-1.5570,  1.1172, -0.7634],
         [-0.2183, -0.7465,  0.4658],
         [-2.1586,  0.6090,  0.3279],
         [ 0.3910,  1.6997, -1.4050],
         [ 0.1179,  0.4655, -0.0590],
         [-0.2183, -0.7465,  0.4658],
         [-0.2208, -1.8266, -1.1779]],

        [[-0.1074,  2.0117, -0.6662],
         [-1.0342,  1.4998, -0.3385],
         [-1.2221,  0.7193,  0.6978],
         [-0.4283, -1.0294,  0.6910],
         [ 0.1350, -0.3478,  0.5812],
         [-0.2183, -0.7465,  0.4658],
         [-0.2208, -1.8266, -1.1779],
         [ 0.8383, -0.2728,  0.8397]]], grad_fn=<EmbeddingBackward0>)


#Positional Encoding

In [ ]:
import math

def positional_encoding(seq_length, embedding_dim):
    PE = []
    for pos in range(seq_length):
        pe_row = []
        for i in range(embedding_dim):
            denom = math.pow(10000, (2 * (i // 2)) / embedding_dim)
            if i % 2 == 0:
                pe_row.append(math.sin(pos / denom))  # Even index -> Sin
            else:
                pe_row.append(math.cos(pos / denom))  # Odd index -> Cos
        PE.append(pe_row)
    return PE


In [ ]:
def add_positional_encoding(embeddings, PE):
    embeddings = embeddings.detach().numpy()  # Convert to NumPy
    print("embedd shape in add_positional_encoding fucntion ",embeddings.shape)
    result = embeddings + np.array(PE)  # Use NumPy broadcasting
    return result.tolist()  # Convert back to a Python list

In [ ]:
PE_eng = positional_encoding(8, embedd_dim) #seq length=8
PE_urdu = positional_encoding(9, embedd_dim) #seq length=11
# Print Results
print("Positional Encoding:")
for row in PE_urdu:
    print(row)


Positional Encoding:
[0.0, 1.0, 0.0]
[0.8414709848078965, 0.5403023058681398, 0.0021544330233656045]
[0.9092974268256817, -0.4161468365471424, 0.0043088560467428125]
[0.1411200080598672, -0.9899924966004454, 0.006463259070189646]
[-0.7568024953079282, -0.6536436208636119, 0.008617632093856952]
[-0.9589242746631385, 0.28366218546322625, 0.010771965118034833]
[-0.27941549819892586, 0.960170286650366, 0.012926248143199044]
[0.6569865987187891, 0.7539022543433046, 0.015080471170057421]
[0.9893582466233818, -0.14550003380861354, 0.017234624199596284]


In [ ]:
eng_posi_embedd = add_positional_encoding(eng_embeddings, PE_eng)
urdu_posi_embedd= add_positional_encoding(urdu_embeddings, PE_urdu)


In [ ]:
print("\nFinal Position-Aware Embeddings:")
for row in eng_posi_embedd:
    print(row)


Final Position-Aware Embeddings:
[[-0.10736947506666183, 3.0117056369781494, -0.6662423014640808], [-0.7155201155033828, 1.6574582154064088, -0.7612612612133898], [0.6910206131993923, -1.1625999429504932, 0.47015181831175745], [-2.0174350227628866, -0.3810121389535033, 0.33435812553671185], [-0.3658361991119443, 1.0460649601071523, -1.3964119031997222], [-0.8410679136912108, 0.7491227839320067, -0.04822000351370955], [-0.49769231182521523, 0.21371718024701514, 0.4787692104082137], [0.43613765095534607, -1.072745265722003, -1.1627789543746814]]
[[-0.10736947506666183, 3.0117056369781494, -0.6662423014640808], [-0.1927641350117446, 2.0401431614665895, -0.3363767630940355], [-0.31284765374072454, 0.30312911483835564, 0.7021153931980795], [-0.28718648212300935, -2.0193842621949276, 0.6974251564433891], [-0.6217785586786329, -1.0014743634611918, 0.5898563191322785], [-1.177201088289428, -0.4627909209401246, 0.4766149273830495], [-0.5002644459623689, -0.8664772334149416, -1.1649331774015397

In [ ]:
#Attention-Mechanism

In [ ]:
import numpy as np

In [ ]:
def flatten_shape(X):
  # Reshape input X to have dimensions (sequence_length, embedding_dim)
    X = np.array(X)
    original_shape = X.shape
    X = X.reshape(-1, original_shape[-1]) # flatten batch and seq_len, keep embedding dim
    return X,original_shape

In [ ]:
def softmax(x):
    exp_x = [math.exp(i) for i in x]  # Compute e^x for each element
    sum_exp_x = sum(exp_x)  # Sum of all exponentials
    return [i / sum_exp_x for i in exp_x]  # divide by sum of all values in x

In [ ]:
def initialize_weights(input_dim, output_dim):
    return np.random.rand(input_dim, output_dim)  # Assign weights between 0 and 1

In [ ]:
def compute_qkv(X):
  # Reshape input X to have dimensions (sequence_length, embedding_dim)
    X,original_shape=flatten_shape(X)

    input_dim = X.shape[1]
    output_dim = input_dim  # Assuming same output dim
    print(f"weight matrix dim :{input_dim}x{output_dim}")
    # Initialize random weights for Q, K, V
    W_q = initialize_weights(input_dim, output_dim)
    W_k = initialize_weights(input_dim, output_dim)
    W_v = initialize_weights(input_dim, output_dim)

    Q = np.dot(X, W_q)
    K = np.dot(X, W_k)
    V = np.dot(X, W_v)

    return Q, K, V,original_shape

In [ ]:
def attention_score(Q,K,V,shape):
    print("shape in ass fn ",shape)
    # Compute dot product scores
    scores = np.dot(Q, np.transpose(K))

    '''
    print("QKt")
    for row in scores:
      print(row)
    '''

    # Scale by sqrt of embedding dimension
    D_k = K.shape[1]  # Get embedding dimension
    scores = scores / math.sqrt(D_k)

    '''
    print("after scaling using Dk")
    for row in scores:
      print(row)
    '''

    # Apply softmax to scores
    scores = np.array([softmax(row) for row in scores])

    '''
    print("after applying softmax")
    for row in scores:
      print(row)
    '''

    # Compute attention output
    output = np.dot(scores, V)
    print("shape in ass fn before changing ",np.shape(output))
    output = np.array(output).reshape(shape)
    print("shape in ass fn after changing ",np.shape(output))
    return output.tolist()

In [ ]:
#testing Q K V
'''
X=[[1,0,1],[0,1,1],[1,1,0]]
WQ = [[1, 0, 1],
      [0, 1, 1],
      [1, 1, 0]]

WK = [[0, 1, 1],
      [1, 0, 1],
      [1, 1, 0]]

WV = [[1, 1, 0],
      [0, 1, 1],
      [1, 0, 1]]

 # Compute Q, K, V
Q, K, V = compute_qkv(X, WQ, WK, WV)
'''
#ascore=attention_score(X,Q,K,V)
eng_input=eng_posi_embedd
ascore_eng=attention_score(eng_input)

for row in ascore_eng:
  print(row)


TypeError: attention_score() missing 2 required positional arguments: 'K' and 'V'

#Add and normalize

In [ ]:
def add_and_normalize(X, attention_output):
    X = np.array(X) # onvert to NumPy array
    attention_output = np.array(attention_output)  # Convert to NumPy array

    added = X + attention_output  # Add input and attention output
    mean = np.mean(added, axis=-1, keepdims=True)
    std = np.std(added, axis=-1, keepdims=True)

    normalized = (added - mean) / (std + 1e-6)  # Normalize
    return normalized.tolist()  # Convert back to list

In [ ]:
attention_output=ascore_eng
embeddings=eng_embeddings

In [ ]:
X = add_and_normalize(embeddings, attention_output)

In [ ]:

for row in X:
  print(row)

[[-1.3766298208525873, 0.40784342043201655, 0.9687864004205702], [-0.6876296470454522, 1.4140346478110042, -0.7264050007655517], [-1.1069072393031676, 1.315728060897963, -0.20882082159479604], [0.8306402690586213, 0.5759030968623944, -1.4065433659210151], [0.2017576759873379, -1.3130929643829559, 1.1113352883956158], [-1.0018142730692678, 1.365356759221646, -0.36354248615237805], [-1.13599838812422, 1.297468869833072, -0.16147048170885211], [0.7727695761511075, 0.6393395585187399, -1.4121091346698476]]
[[-1.3766298208525873, 0.40784342043201655, 0.9687864004205702], [-1.403598061231566, 0.5520264320031492, 0.8515716292284172], [-1.1069072393031676, 1.315728060897963, -0.20882082159479604], [0.7704822476285417, -1.4122551600875772, 0.6417729124590356], [0.6766507821241936, -1.4137783646509525, 0.7371275825267589], [-1.1071691728990365, 1.3155736407913157, -0.2084044678922788], [0.8170533616207053, 0.591127267989268, -1.4081806296099744], [-1.364217005925197, 0.35933823674987153, 1.00487

#Feed Forward Network

In [ ]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedForwardNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
original_shape=flatten_shape(X)

In [ ]:
print(X.shape)

(16, 3)


In [ ]:
ffn = FeedForwardNetwork(input_dim=3, hidden_dim=4, output_dim=3)
torch_input = torch.tensor(X, dtype=torch.float32)
ffn_output = ffn(torch_input).detach().numpy().tolist()


In [ ]:
for row in ffn_output:
  print(row)

[-0.014715135097503662, 0.2408999353647232, -0.8444631099700928]
[0.22484225034713745, 0.49532079696655273, -0.5651528835296631]
[0.13842114806175232, 0.4131852984428406, -0.6645765900611877]
[0.4395514726638794, 0.6987361907958984, -0.31795191764831543]
[0.4961879849433899, 0.24676181375980377, -0.8073462247848511]
[0.16213847696781158, 0.43574005365371704, -0.6372947692871094]
[0.13148394227027893, 0.4065859317779541, -0.6725557446479797]
[0.43395814299583435, 0.6934619545936584, -0.32439887523651123]
[-0.014715135097503662, 0.2408999353647232, -0.8444631099700928]
[0.005203664302825928, 0.2721821665763855, -0.8197773694992065]
[0.13842114806175232, 0.4131852984428406, -0.6645765900611877]
[0.7144808769226074, 0.3626060485839844, -0.6902244687080383]
[0.6888234615325928, 0.3488425016403198, -0.7074888944625854]
[0.13835948705673218, 0.41312670707702637, -0.6646474599838257]
[0.43826547265052795, 0.6975239515304565, -0.3194342851638794]
[-0.020381629467010498, 0.23174339532852173, -0.

In [ ]:
original_shape=None

#Masking Mechanism

In [ ]:
def apply_mask(X):
    for batch in X:
        for i in range(len(batch)):
            for j in range(len(batch[i])):
                if j > i:
                    batch[i][j] = 0
    return X

#Encoder

In [ ]:
def encoder(X_embedd,X_posi_embedd):
    # Compute Q, K, V
    Q, K, V,shape = compute_qkv(X_posi_embedd)
    #computing attention score
    attention_output = attention_score(Q, K, V,shape)
    print("attention output shape ",np.shape(attention_output))
    #add and normalize 1
    norm_output1 = add_and_normalize(X_embedd, attention_output)
    #passing through feed forward,fist changing list to tensor
    ffn = FeedForwardNetwork(input_dim=3, hidden_dim=4, output_dim=3)
    torch_input = torch.tensor(norm_output1, dtype=torch.float32)
    ffn_output = ffn(torch_input).detach().numpy().tolist()
    #add and normalize 2
    norm_output2=add_and_normalize(norm_output1, ffn_output)
    return norm_output2


#Decoder

In [ ]:
def decoder(X_embedd,X_posi_embedd, encoder_K,encoder_V):

    #Compute Q, K, V
    Q, K, V,shape = compute_qkv(X_posi_embedd)
    #computing attention score
    attention_output = attention_score(Q, K, V,shape)#here it return attention score of 2,9,3
    #store the masked attention now
    mask_attention_output=apply_mask(attention_output)
    #add and normalize 1
    norm_output1 = add_and_normalize(X_embedd, mask_attention_output)
    decoder_Q,_,_,d_shape = compute_qkv(norm_output1)
    #cross attention
    cross_attention_output = attention_score(decoder_Q, encoder_K, encoder_V,d_shape)
    #add and normalize 2
    norm_output2 = add_and_normalize(norm_output1, cross_attention_output)
    #passing through feed forward,fist changing list to tensor
    ffn = FeedForwardNetwork(input_dim=3, hidden_dim=4, output_dim=3)
    torch_input = torch.tensor(norm_output2, dtype=torch.float32)
    ffn_output = ffn(torch_input).detach().numpy().tolist()
    #add and normalize 3
    norm_output3=add_and_normalize(norm_output2, ffn_output)
    return norm_output3



#Forward

In [48]:
def transformer_forward(eng_embedd,eng_posi_embedd,urdu_embedd,urdu_posi_embedd):
    encoder_output = encoder(eng_embedd,eng_posi_embedd)
    _, K, V,_ = compute_qkv(final_encoder_output)
    decoder_output = decoder(urdu_embedd,urdu_posi_embedd,K,V)
    #Linear Layer
    decoder_output = torch.tensor(decoder_output, dtype=torch.float32)
    final_layer = nn.Linear(3, 3)
    final_output = final_layer(decoder_output)
    #Softmax
    softmax_output = np.apply_along_axis(softmax, -1, final_output.detach().numpy())
    print(softmax_output.shape)
    return softmax_output

In [41]:
eng_embedd=eng_embeddings.tolist()
urdu_embedd=urdu_embeddings.tolist()

In [49]:
output_prob=transformer_forward(eng_embedd,eng_posi_embedd,urdu_embedd,urdu_posi_embedd)
output_prob

weight matrix dim :3x3
shape in ass fn  (2, 8, 3)
shape in ass fn before changing  (16, 3)
shape in ass fn after changing  (2, 8, 3)
attention output shape  (2, 8, 3)
weight matrix dim :3x3
weight matrix dim :3x3
shape in ass fn  (2, 9, 3)
shape in ass fn before changing  (18, 3)
shape in ass fn after changing  (2, 9, 3)
weight matrix dim :3x3
shape in ass fn  (2, 9, 3)
shape in ass fn before changing  (18, 3)
shape in ass fn after changing  (2, 9, 3)
(2, 9, 3)


array([[[0.58888962, 0.106662  , 0.30444838],
        [0.40466756, 0.17118553, 0.42414691],
        [0.3715934 , 0.19513677, 0.43326983],
        [0.16397971, 0.53987771, 0.29614258],
        [0.29842754, 0.26439413, 0.43717834],
        [0.44004464, 0.14955597, 0.41039939],
        [0.55264931, 0.15450672, 0.29284398],
        [0.16825335, 0.49182029, 0.33992636],
        [0.42072963, 0.28229682, 0.29697354]],

       [[0.58888962, 0.106662  , 0.30444838],
        [0.49768778, 0.12190251, 0.38040971],
        [0.31652674, 0.24481386, 0.4386594 ],
        [0.20443446, 0.40473761, 0.39082793],
        [0.50168008, 0.20345694, 0.29486299],
        [0.56541803, 0.14161569, 0.29296628],
        [0.5653169 , 0.10195141, 0.33273168],
        [0.16825335, 0.49182029, 0.33992636],
        [0.42072963, 0.28229682, 0.29697354]]])

In [51]:
output_prob_tensor = torch.tensor(output_prob)  # Convert list to tensor

# 1. Check sum of probabilities (should be close to 1)
prob_sums = output_prob_tensor.sum(dim=-1)
print("Sum of probabilities per token:\n", prob_sums)

# 2. Get predicted token indices (argmax)
predicted_tokens = output_prob_tensor.argmax(dim=-1)
print("Predicted token indices:\n", predicted_tokens)

# 3. Check shape
print("Output shape:", output_prob_tensor.shape)

Sum of probabilities per token:
 tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],
       dtype=torch.float64)
Predicted token indices:
 tensor([[0, 2, 2, 1, 2, 0, 0, 1, 0],
        [0, 0, 2, 1, 0, 0, 0, 1, 0]])
Output shape: torch.Size([2, 9, 3])


In [52]:
id_to_word = {v: k for k, v in tokenizer.get_vocab().items()}


In [53]:
def decode_tokens(predicted_indices, id_to_word):
    sentences = []
    for sentence in predicted_indices:
        words = [id_to_word[idx] for idx in sentence]
        sentences.append(" ".join(words))  # Join words to form a sentence
    return sentences


In [55]:
predicted_sentences = decode_tokens(predicted_tokens.tolist(), id_to_word)
for i, sent in enumerate(predicted_sentences):
    print(f"Predicted Sentence {i+1}: {sent}")

Predicted Sentence 1: <s> </s> </s> <pad> </s> <s> <s> <pad> <s>
Predicted Sentence 2: <s> <s> </s> <pad> <s> <s> <s> <pad> <s>
